In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [18]:
x_test=np.load('../Data/x_test.npy')
y_test=np.load('../Data/y_test.npy')
print(x_test.shape, y_test.shape)

(3600, 2) (3600,)


In [19]:
model=load_model('../NN_model/15-0.9637.h5', compile=False)
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 8)                 24        
                                                                 
 dense_22 (Dense)            (None, 4)                 36        
                                                                 
 dense_23 (Dense)            (None, 1)                 5         
                                                                 
Total params: 65 (260.00 Byte)
Trainable params: 65 (260.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if y_pred[i]>=0.5:
            count1+=1
print(count,count1)

## HLS4ML

In [ ]:
import os
import hls4ml
import plotting
from hls4ml.model import profiling

In [ ]:
# os.environ['PATH']+= os.pathsep + '/opt/Xilinx/Vivado/2022.1/bin'
os.environ['PATH']+= os.pathsep + '/opt/Xilinx/Vivado/2018.3/bin'

In [ ]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name')
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")

In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='../model_2/myproject_prj',part='xc7vx485tffg1761-2',
)

In [ ]:
hls_model.compile()
y_hls=hls_model.predict(np.ascontiguousarray(x_test))

In [ ]:
count=0
count1=0
for i in range(y_hls.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_hls[i]<=0.5:
            count1+=1
print(count,count1)

In [ ]:
hls_model.build()

In [ ]:
hls4ml.report.read_vivado_report('../model_2/myproject_prj')